In [1]:
# ----------------------------------------------------------------------
# BƯỚC 1: KẾT NỐI GOOGLE DRIVE
# ----------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

drive  sample_data


In [3]:
# !ls nnUNet_Workspace/nnUNet/

In [3]:
# ----------------------------------------------------------------------
# BƯỚC 2: CÀI ĐẶT MÔI TRƯỜNG & GIẢI NÉN CODE
# ----------------------------------------------------------------------
import os
import shutil

In [4]:
# Tạo thư mục làm việc
!mkdir -p /content/nnUNet_Workspace
%cd /content/nnUNet_Workspace

# Copy và giải nén Code (Giả sử bạn để file zip ở thư mục gốc Drive/BraTS2020_EDL_Project)
!cp "/content/drive/MyDrive/XUM_project/nnUNet.zip" .
!unzip -q nnUNet.zip
!rm nnUNet.zip

# Cài đặt thư viện từ source code vừa giải nén
%cd nnUNet
!pip install -e .
!pip install --upgrade hiddenlayer graphviz # Cài thêm visualizer nếu cần

/content/nnUNet_Workspace
/content/nnUNet_Workspace/nnUNet
Obtaining file:///content/nnUNet_Workspace/nnUNet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [1]:
!ls

drive  nnUNet_Workspace  sample_data


## Giải nén `nnUNet_preprocessed` vào thư mục content của Colab

In [2]:
import os
import zipfile
from tqdm import tqdm
import shutil

# ----------------------------------------------------------------------
# BƯỚC 3: GIẢI NÉN DỮ LIỆU PREPROCESSED – AUTO FIX FOLDER DOUBLE-LAYER
# ----------------------------------------------------------------------

os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = "/content/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/content/drive/MyDrive/XUM_project/nnUNet_results"

!mkdir -p $nnUNet_raw
!mkdir -p $nnUNet_preprocessed

zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_preprocessed.zip"
zip_local = "/content/nnUNET_preprocessed.zip"
extract_root = "/content/nnUNet_preprocessed"

print("📦 Copy file từ Drive…")
!cp "$zip_path_drive" "$zip_local"

print("💨 Copy xong. Bắt đầu giải nén…")

with zipfile.ZipFile(zip_local, 'r') as zip_ref:
    # Detect folder root name inside zip (vd: nnUNET_preprocessed/)
    root_name = zip_ref.namelist()[0].split('/')[0]

    file_list = zip_ref.infolist()
    total_files = len(file_list)

    print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
    print("🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…")

    for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping"):
        file_path = file.filename

        # Bỏ lớp folder đầu tiên
        fixed_path = file_path.replace(root_name + "/", "", 1)

        if fixed_path.strip() == "":
            continue

        # Chỉ định nơi lưu đúng
        target_path = os.path.join(extract_root, fixed_path)

        # Tạo thư mục nếu cần
        if file.is_dir():
            os.makedirs(target_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(target_path), exist_ok=True)
            with zip_ref.open(file) as source, open(target_path, "wb") as target:
                target.write(source.read())

os.remove(zip_local)

print("🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥")

📦 Copy file từ Drive…
💨 Copy xong. Bắt đầu giải nén…
🔧 Phát hiện folder root trong ZIP: nnUNET_preprocessed
🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…


🔄 Unzipping: 100%|██████████| 2584/2584 [03:09<00:00, 13.64it/s]


🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥


## Giải nén `nnUNet_raw` vào thư mục content của Colab

In [3]:
!ls /content/drive/MyDrive/XUM_project

inference_results	 nnUNET_raw.zip  run_batch_inference.py
inference_results_batch  nnUNet_results  Train_BraTS_EDL.ipynb
nnUNET_preprocessed.zip  nnUNet.zip


In [4]:
import os

print("Exists? ->", os.path.exists("/content/drive/MyDrive/XUM_project/nnUNET_raw.zip"))
print(os.listdir("/content/drive/MyDrive/XUM_project"))


Exists? -> True
['nnUNET_preprocessed.zip', 'nnUNET_raw.zip', 'nnUNet_results', 'nnUNet.zip', 'inference_results', '.ipynb_checkpoints', 'inference_results_batch', 'run_batch_inference.py', 'Train_BraTS_EDL.ipynb']


In [5]:
import os
import zipfile
from tqdm import tqdm
import shutil

!pwd

# ----------------------------------------------------------------------
# BƯỚC: GIẢI NÉN DỮ LIỆU RAW (QUAN TRỌNG CHO INFERENCE)
# ----------------------------------------------------------------------

# 1. Thiết lập đường dẫn (Đảm bảo biến môi trường vẫn đúng)
os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
!mkdir -p $nnUNet_raw

# Đường dẫn file zip trên Drive (Bạn kiểm tra lại tên file cho đúng nhé)
# Giả sử bạn lưu là nnUNet_raw.zip trong thư mục XUM_project
zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_raw.zip"
zip_local = "/content/nnUNet_raw.zip"
extract_root = "/content/nnUNet_raw"

if not os.path.exists(zip_path_drive):
    print(f"❌ Lỗi: Không tìm thấy file {zip_path_drive}")
    print("👉 Hãy kiểm tra lại tên file zip raw trên Google Drive của bạn.")
else:
    print("📦 Copy file Raw Data từ Drive...")
    !cp "$zip_path_drive" "$zip_local"

    print("💨 Copy xong. Bắt đầu giải nén Raw Data...")

    with zipfile.ZipFile(zip_local, 'r') as zip_ref:
        # Detect folder root name inside zip (ví dụ: nnUNet_raw/)
        # Để tránh bị lồng thư mục kiểu /content/nnUNet_raw/nnUNet_raw/Dataset...
        root_name = zip_ref.namelist()[0].split('/')[0]

        file_list = zip_ref.infolist()
        total_files = len(file_list)

        print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
        print("🔧 Đang giải nén và loại bỏ lớp folder thừa...")

        for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping Raw"):
            file_path = file.filename

            # Logic: Nếu file nằm trong folder root, ta bỏ tên root đi để file nhảy ra ngoài
            # Ví dụ: nnUNet_raw/Dataset101/abc.nii -> Dataset101/abc.nii
            if file_path.startswith(root_name + "/"):
                fixed_path = file_path.replace(root_name + "/", "", 1)
            else:
                fixed_path = file_path # Nếu nén không có folder root thì giữ nguyên

            if fixed_path.strip() == "":
                continue

            # Chỉ định nơi lưu đúng
            target_path = os.path.join(extract_root, fixed_path)

            # Tạo thư mục nếu cần
            if file.is_dir():
                os.makedirs(target_path, exist_ok=True)
            else:
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                with zip_ref.open(file) as source, open(target_path, "wb") as target:
                    target.write(source.read())

    os.remove(zip_local)
    print("🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀")

/content
📦 Copy file Raw Data từ Drive...
💨 Copy xong. Bắt đầu giải nén Raw Data...
🔧 Phát hiện folder root trong ZIP: nnUNET_raw
🔧 Đang giải nén và loại bỏ lớp folder thừa...


🔄 Unzipping Raw: 100%|██████████| 1845/1845 [04:43<00:00,  6.50it/s]


🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀


In [6]:
# ----------------------------------------------------------------------
# BƯỚC 4: KIỂM TRA BIẾN MÔI TRƯỜNG & DATA
# ----------------------------------------------------------------------
print(f"Raw: {os.environ['nnUNet_raw']}")
print(f"Preprocessed: {os.environ['nnUNet_preprocessed']}")
print(f"Results: {os.environ['nnUNet_results']}")

# Kiểm tra file EDLTrainer có tồn tại không
if os.path.exists("/content/nnUNet_Workspace/nnUNet/nnunetv2/training/nnUNetTrainer/EDLTrainer.py"):
    print("✅ Tim thay file EDLTrainer.py!")
else:
    print("❌ KHONG TIM THAY EDLTrainer.py! Kiem tra lai file zip code.")

Raw: /content/nnUNet_raw
Preprocessed: /content/nnUNet_preprocessed
Results: /content/drive/MyDrive/XUM_project/nnUNet_results
✅ Tim thay file EDLTrainer.py!


In [ ]:
# !cat /content/nnUNet_Workspace/nnUNet/nnunetv2/training/nnUNetTrainer/EDLTrainer.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from nnunetv2.training.nnUNetTrainer.nnUNetTrainer import nnUNetTrainer
from nnunetv2.training.loss.dice import MemoryEfficientSoftDiceLoss
from nnunetv2.training.loss.deep_supervision import DeepSupervisionWrapper

# --- PHẦN 1: HÀM LOSS EDL CHUẨN HOÁ ---
class EDLLoss(nn.Module):
    def __init__(self, num_classes, annealing_step=10, lamb=1.0):
        super(EDLLoss, self).__init__()
        self.num_classes = num_classes
        self.annealing_step = annealing_step
        self.lamb = lamb
        
        # Biến này sẽ được update từ Trainer mỗi epoch
        self.current_epoch = 0 
        
        # Dice Loss: tắt do_bg (nếu cần), batch_dice=True để ổn định
        self.dice_loss = MemoryEfficientSoftDiceLoss(batch_dice=True, do_bg=False, smooth=1e-5, ddp=False)

    def KL(self, alpha):
        # KL Divergence: Dirichlet(alpha) || Dirichlet(1)
        beta = torch.ones((1, self.num_classes) + a

In [13]:
!ls nnUNet_preprocessed -a

.  ..  Dataset101_BraTS2020


### Training bằng `EDLTrainer`

In [ ]:
# ----------------------------------------------------------------------
# BƯỚC 5: BẮT ĐẦU TRAINING (The Moment of Truth)
# ----------------------------------------------------------------------
# Lệnh train:
# Dataset101_BraTS2020: Tên dataset
# 3d_fullres: Cấu hình
# 0: Fold (thường chạy fold 0 trước)
# -tr EDLTrainer: Dùng Trainer Custom của bạn
# --npz: Lưu xác suất softmax (để dùng cho validation sau này)

# Khai báo biến môi trường trực tiếp trên cùng 1 dòng lệnh để đảm bảo nnU-Net nhận diện được
!nnUNet_raw="/content/nnUNet_raw" \
nnUNet_preprocessed="/content/nnUNet_preprocessed" \
nnUNet_results="/content/drive/MyDrive/XUM_project/nnUNet_results" \
nnUNetv2_train Dataset101_BraTS2020 3d_fullres 0 -tr EDLTrainer --npz

# Sao lưu file split quan trọng về Drive để dùng cho Inference sau này
!cp "/content/nnUNet_preprocessed/Dataset101_BraTS2020/splits_final.json" "/content/drive/MyDrive/BraTS2020_EDL_Project/"

print("✅ Đã backup splits_final.json về Drive an toàn!")


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-11-22 13:01:22.781290: Using torch.compile...
2025-11-22 13:01:27.235220: do_dummy_2d_data_aug: False
2025-11-22 13:01:27.240783: Creating new 5-fold cross-validation split...
2025-11-22 13:01:27.247902: Desired fold for training: 0
2025-11-22 13:01:27.252467: This split has 294 training an

### Copy `dataset.json` từ `nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres` vô `nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres/fold_0`

In [7]:
# !python inference_edl.py

python3: can't open file '/content/inference_edl.py': [Errno 2] No such file or directory


### Demo testing

In [10]:
import os
import json
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import save_json

def regenerate_splits():
    # 1. Cấu hình đường dẫn
    preprocessed_dir = "/content/nnUNet_preprocessed/Dataset101_BraTS2020"
    splits_file = os.path.join(preprocessed_dir, "splits_final.json")
    gt_folder = os.path.join(preprocessed_dir, "gt_segmentations")

    if os.path.exists(splits_file):
        print("✅ File splits_final.json đã tồn tại. Không cần tạo lại.")
        return

    print("⚠️ Không tìm thấy splits_final.json. Đang tái tạo lại theo chuẩn nnU-Net...")

    # 2. Lấy danh sách tất cả Case ID từ thư mục Ground Truth đã preprocess
    if not os.path.exists(gt_folder):
        raise FileNotFoundError(f"❌ Lỗi: Không tìm thấy folder {gt_folder}. Bạn đã giải nén data chưa?")

    files = sorted([f for f in os.listdir(gt_folder) if f.endswith(".nii.gz") or f.endswith(".nii")])
    identifiers = [f.replace(".nii.gz", "").replace(".nii", "") for f in files]

    print(f"📂 Tìm thấy {len(identifiers)} ca bệnh trong tập dữ liệu.")

    # 3. Tái tạo Split (Sử dụng thuật toán K-Fold của nnU-Net)
    # Code này mô phỏng lại hàm generate_crossval_split của nnU-Net
    seed = 12345
    n_splits = 5

    kfold = []

    # Xáo trộn danh sách với seed cố định để đảm bảo giống hệt lần train trước
    rs = np.random.RandomState(seed)
    keys = np.unique(identifiers)
    rs.shuffle(keys)

    # Chia k-fold
    k = n_splits
    for i in range(k):
        # Logic chia: fold i sẽ là tập val, các fold còn lại là train
        val_keys = keys[i::k]
        train_keys = [x for x in keys if x not in val_keys]

        kfold.append({
            'train': list(train_keys),
            'val': list(val_keys)
        })

    # 4. Lưu file
    save_json(kfold, splits_file)
    print(f"🎉 Đã tạo xong file: {splits_file}")
    print("👉 Bây giờ bạn có thể chạy lại file run_batch_inference.py!")

if __name__ == "__main__":
    regenerate_splits()

⚠️ Không tìm thấy splits_final.json. Đang tái tạo lại theo chuẩn nnU-Net...
📂 Tìm thấy 368 ca bệnh trong tập dữ liệu.
🎉 Đã tạo xong file: /content/nnUNet_preprocessed/Dataset101_BraTS2020/splits_final.json
👉 Bây giờ bạn có thể chạy lại file run_batch_inference.py!


In [ ]:
!python "/content/drive/MyDrive/XUM_project/run_batch_inference.py"

🚀 --- BATCH INFERENCE STARTED ---
📂 Model Loaded from: /content/drive/MyDrive/XUM_project/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres
📂 Found total 368 cases in raw folder.
📂 Đã load danh sách Validation Fold 0: 74 ca.
⚙️ Mode: VALIDATION SPLIT (Fold 0) -> Running 74 cases.

🔍 Processing: BRATS_039...
100% 4/4 [00:08<00:00,  2.12s/it]
    📸 Drawing Slice: 96
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_batch/BRATS_039_slice96.png

🔍 Processing: BRATS_153...
100% 8/8 [00:16<00:00,  2.07s/it]
    📸 Drawing Slice: 109
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_batch/BRATS_153_slice109.png

🔍 Processing: BRATS_299...
100% 8/8 [00:16<00:00,  2.10s/it]
    📸 Drawing Slice: 103
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_batch/BRATS_299_slice103.png

🔍 Processing: BRATS_155...
100% 8/8 [00:17<00:00,  2.16s/it]
    📸 Drawing Slice: 87
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_b

In [44]:
# !python "/content/drive/MyDrive/XUM_project/run_batch_inference.py"

🚀 --- BATCH INFERENCE STARTED ---
📂 Model Loaded from: /content/drive/MyDrive/XUM_project/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres
📂 Found total 368 cases in raw folder.
⚙️ Mode: RANGE [186:190] -> Running 4 cases.

🔍 Processing: BRATS_187...
100% 4/4 [00:08<00:00,  2.06s/it]
    📸 Drawing Slice: 38
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_batch/BRATS_187_slice38.png

🔍 Processing: BRATS_188...
100% 8/8 [00:15<00:00,  2.00s/it]
    📸 Drawing Slice: 46
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_batch/BRATS_188_slice46.png

🔍 Processing: BRATS_189...
100% 8/8 [00:16<00:00,  2.05s/it]
    📸 Drawing Slice: 54
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_batch/BRATS_189_slice54.png

🔍 Processing: BRATS_190...
100% 8/8 [00:16<00:00,  2.05s/it]
    📸 Drawing Slice: 117
    ✅ Saved: /content/drive/MyDrive/XUM_project/inference_results_batch/BRATS_190_slice117.png

✅ --- BATCH INFERENCE FINISHED 